# Produção automática de mapas - Um mapa por ficheiro (1) #

**Nota**: Este programa foi usado no Projecto do Caminho de Fátima do Centenário

O seguinte programa produz, para cada _Feature_ _Class_ numa pasta, um novo mapa a partir de um Template Base (.MXD). De mapa para mapa, o procedimento garante que a extensão geográfica do novo mapa corresponderá à extensão de cada _Feature_ _Class_ na dita pasta. Posto isto, o Template Base deve incluir toda a informação a representar em cada mapa, o programa limitar-se-á a destacar a área coberta pela extensão geográfica de cada _Feature_ _Class_.

### Parâmetros: ###

* **MXD_TEMPLATE** -> Caminho para o Template Base (ficheiro MXD);

* **PASTA_SHP** -> Caminho para a pasta onde se encontram as _Feature_ _Classes_ que vão ser usadas para circunscrever a área geográfica de cada mapa;

* **DATAFRAME_PRINCIPAL** -> Nome do Dataframe do ArcMap com informação relevante, aquele em que se vai registar a mudança de extensão geográfica;

* **NOME_LAYER_EXTENT** -> Nome da Layer do ArcMap que deverá assumir a extensão de cada _Feature_ _Class_ na **PASTA_SHP**;

* **DATAFRAMES_CONTEXTO** -> Dicionário com referências a outros Dataframes do ArcMap que deverão ter a _Data_ _Source_ de algumas _Layers_ modificadas de mapa para mapa.

```Python
contextDataFrames = {
    "nome_dataframe_que_tambem_deve_ser_modificado" : {
        "PATHDATA" : r'/path/to/data', # Caminho para uma pasta com os dados que deverão ser usados como data source das layers deste dataframe
        "LAYERS"   : {
            # Aqui, o nome_campo é o nome de um campo de cada uma das Feat. Class dentro da PASTA_SHP.
            # O valor do campo indica o nome do ficheiro (que se encontra em r'/path/to/data') que deve ser usado como
            # Data Source da "layer_deste_dataframe_a_modificar
            "layer_deste_dataframe_a_modificar" : "nome_campo",
            # Se o objectivo for substituir a fonte de dados de uma layer deste dataframe pelas Feat.Class
            # dentro da PASTA_SHP, deve-se usar a str "EXTENTFILE"
            "layer_deste_dataframe_a_modificar2" : "EXTENTFILE"
        },
        # A key "ISZOOM" é opcional. Se existir, neste Dataframe, a layer associada a este Key deve definir a extensão geográfica do Dataframe
        "ISZOOM" : "layer_deste_dataframe_a_modificar",
        # A key "EXTENT_TOLERANCE" é opcional e pode ser usada para exagerar/alargar/aumentar a extensão da área geográfica que será atribuída ao dataframe (este valor numérico em metros será adicionada à extensão da Layer que é valor da key "ISZOOM"). Esta key só será considerada se a key "ISZOOM" existir no dicionário.
        "EXTENT_TOLERANCE" : 1000 # Metros
    }
}
```

* **ELEMENTOS_GRAFICOS** -> Tabela Excel (.xlsx) na qual se indicam os valores a atribuir a determinados elementos gráficos do tipo texto num mapa específico.
    * O ficheiro não pode ter mais que um _Sheet_.

|                    | text_element_1 | text_element_2 | text_element_n |
| ------------------ | -------------- | -------------- | -------------- |
| PASTA_SHAPE_FILE_1 | valor_a_usar   | valor_a_usar   | valor_a_usar   |
| PASTA_SHAPE_FILE_2 | valor_a_usar   | valor_a_usar   | valor_a_usar   |
| PASTA_SHAPE_FILE_3 | valor_a_usar   | valor_a_usar   | valor_a_usar   |
| PASTA_SHAPE_FILE_N | valor_a_usar   | valor_a_usar   | valor_a_usar   |

* **PASTA_MXD** -> Caminho para a pasta onde serão guardados os MXD resultantes da execução do programa;
* **PASTA_MAPS** -> Caminho para a pasta onde serão guardados os Mapas resultantes da execução do programa;
* **FORMATO_MAPS** -> Formato dos mapas de saída (e.g. pdf, jpg, tiff, png, eps).

### Requisitos: ###
* **GIS Software:**
    * ArcGIS.

* **Python Packages:**
    * Pandas;
    * XlsxWriter;
    * ArcPy.

In [ ]:
from glass.mapseries.fmfile import maps_by_extent

MXD_TEMPLATE        = r'D:\TRENMO_JASP\SITIOS_FATIMA\acf_maps_versao_final\main_caminho.mxd'
PASTA_SHP           = r'D:\TRENMO_JASP\tst_script\maps_production\map_cells'
DATAFRAME_PRINCIPAL = 'main'
NOME_LAYER_EXTENT   = 'map_ext'
DATAFRAMES_CONTEXTO = {
    "zoom_pt" : {
        "PATHDATA" : r'D:\TRENMO_JASP\tst_script\maps_production\lmt_conc',
        "LAYERS"   : {
            "ref_concelho" : "concelho"
        }
    },
    "zoom_concelho" : {
        "PATHDATA" : r'D:\TRENMO_JASP\tst_script\maps_production\lmt_conc',
        "LAYERS"   : {
            "ref_concelho" : "concelho",
            "ref_grid"     : "EXTENTFILE"
        },
        "ISZOOM"   : "ref_concelho",
        "EXTENT_TOLERANCE" : 1000 # in meters
    }
}
ELEMENTOS_GRAFICOS  = r'D:\TRENMO_JASP\tst_script\maps_production\extent_graph.xlsx'
PASTA_MXD           = r'D:\TRENMO_JASP\tst_script\maps_production\mxd'
PASTA_MAPS          = r'D:\TRENMO_JASP\tst_script\maps_production\maps'
FORMATO_MAPS        = '.jpg'

maps_by_extent(
    PASTA_SHP, MXD_TEMPLATE, DATAFRAME_PRINCIPAL, NOME_LAYER_EXTENT,
    DATAFRAMES_CONTEXTO, ELEMENTOS_GRAFICOS, PASTA_MXD, outputMaps=PASTA_MAPS,
    mapsFormat=FORMATO_MAPS
)

# Produção automática de mapas - Um mapa por ficheiro(s) (2) #

**Nota**: Este programa foi usado para gerar automaticamente um mapa para cada Linha de Autocarros em Cascais.

Imagine-se que é necessário produzir um mapa para cada uma das linhas de um determinado operador de transportes. Em cada mapa, para além da geometria da Linha, deve ser também apresentada a distribuição geográfica das paragens. Neste cenário e tendo por objectivo a produção automática de todos os mapas a partir de um template base (ficheiro .mxd), construiu-se um programa capaz de modificar as duas layers, garantindo que em cada cartograma estejam as paragens da Linha a ser representada. Para tal, este procedimento depende de uma tabela em que se indicam que temas (Linhas/Paragens) devem aparecer juntas no mesmo mapa:

Tabela I:

|       | layer_linhas | layer_pontos |
| ----- | ------------ | ------------ |
| map_1 | linha_1      | paragens_1   |
| map_2 | linha_2      | paragens_2   |
| map_3 | linha_3      | paragens_3   |
| map_4 | linha_4      | paragens_4   |

**Nota:** O que foi apresentado é só um exemplo, não há limite para o número de _Layers_ que se podem modificar entre mapas. É tudo uma questão de se adicionarem mais colunas à Tabela.

### Parâmetros: ###

* **MXD_TEMPLATE** -> Caminho para o Template Base (ficheiro MXD);

* **LAYERS_EM_MAP** -> Caminho para uma Tabela Excel (.xlsx) equivalente à Tabela I;
    * O ficheiro não pode ter mais que um _Sheet_.

* **DATAFRAME_NAME** -> Nome do Dataframe no qual as layers devem ser alteradas;

* **LAYERS_DATA_SOURCE** -> Dicionário com a associação entre as LAYERS a alterar e as pastas onde se encontram os ficheiros que serão usados como _Data_ _Source_ em cada mapa;

```Python
LAYERS_DATA_SOURCE = {
    "layer_linhas" : r'/path/to/layer_linhas/data,
    "layer_pontos" : r'/path/to/layer_pontos/data
}
```

* **ELEMENTOS_GRAFICOS** -> Tabela Excel (.xlsx) na qual se indicam os valores a atribuir a determinados elementos gráficos do tipo texto num mapa específico;
    * O ficheiro não pode ter mais que um _Sheet_.

Tabela II:

|       | text_element_1 | text_element_2 | text_element_n |
| ----- | -------------- | -------------- | -------------- |
| map_1 | valor_a_usar   | valor_a_usar   | valor_a_usar   |
| map_2 | valor_a_usar   | valor_a_usar   | valor_a_usar   |
| map_3 | valor_a_usar   | valor_a_usar   | valor_a_usar   |
| map_4 | valor_a_usar   | valor_a_usar   | valor_a_usar   |

**Nota**: A primeira coluna da Tabela I e II dizem respeito à mesma coisa, ou seja, os valores a atribuir aos elementos do layout do tipo texto no "map_1" sê-lo-ão no mesmo mapa em que serão representadas as _layers_ "linha_1" e "paragens_1".

* **LAYER_EXTENSAO** -> Nome de uma _Layers_ que vai sendo alterada de mapa para mapa e que deve determinar a extensão geográfica do mapa;

* **PASTA_MXD** -> Caminho para a pasta onde serão guardados os MXD resultantes da execução do programa;
* **PASTA_MAPS** -> Caminho para a pasta onde serão guardados os Mapas resultantes da execução do programa;
* **FORMATO_MAPS** -> Formato dos mapas de saída (e.g. pdf, jpg, tiff, png, eps);
* **EXTENT_TOLERANCE** -> Tolerância em metros que alargará/aumentará/exagerá a extensão da **LAYER_EXTENSAO**. 


### Requisitos: ###
* **GIS Software:**
    * ArcGIS.

* **Python Packages:**
    * Pandas;
    * XlsxWriter;
    * ArcPy.

In [ ]:
from glass.mapseries.fmfile import files_to_map

MXD_TEMPLATE       = r'D:\TRENMO_JASP\tst_script\maps_production\excascais\base_project2.mxd'
LAYERS_EM_MAP      = r'D:\TRENMO_JASP\tst_script\maps_production\excascais\layers_and_files.xlsx'
DATAFRAME_NAME     = 'Layers'
LAYERS_DATA_SOURCE = {
    'pontos' : r'D:\TRENMO_JASP\tst_script\maps_production\excascais\paragens_etrs',
    'linhas' : r'D:\TRENMO_JASP\tst_script\maps_production\excascais\linhas_etrs'
}
ELEMENTOS_GRAFICOS = r'D:\TRENMO_JASP\tst_script\maps_production\excascais\Titulo_Mapa_25062018.xlsx'
LAYER_EXTENSAO     = 'linhas'
PASTA_MXD          = r'D:\TRENMO_JASP\tst_script\maps_production\excascais\mxd'
PASTA_MAPS         = r'D:\TRENMO_JASP\tst_script\maps_production\excascais\maps'
FORMATO_MAPS       = '.jpg'
EXTENT_TOLERANCE   = 200

files_to_map(
    MXD_TEMPLATE, LAYERS_EM_MAP, DATAFRAME_NAME, LAYERS_DATA_SOURCE, ELEMENTOS_GRAFICOS,
    PASTA_MXD, exportMap=PASTA_MAPS, lyr_extent_name=LAYER_EXTENSAO,
    maps_format=FORMATO_MAPS, extentTolerance=EXTENT_TOLERANCE
)